In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Religious Music,Bright And Joyful Is The Morn,Bright and joyful is the morn \r\nFor us to a...
1,Alice In Chains,Whatcha Gonna Do,Hey now baby whatcha gonna do when I go? \r\n...
2,Robbie Williams,A Man For All Seasons,"One eye on the shadows, protecting his fellows..."
3,Townes Van Zandt,Snowin On Raton,When the wind don't blow in Amarillo \r\nAnd ...
4,Chaka Khan,I Got The Right Street,I got the right street \r\nBut the wrong dire...
5,Weezer,Freak Me Out,City streets at night \r\nCan be so intimidat...
6,Offspring,Long Way Home,I'm driving down a dusty road \r\nI've got no...
7,George Jones,Fiddle And Guitar Band,"Hey, I've got a guitar, I've got a fiddle \r\..."
8,Zayn Malik,PillowTalk,[Verse 1] \r\nClimb on board \r\nWe'll go sl...
9,Zao,Breath Of The Black Muse,"Open your mouth, boy \r\nJust to swallow your..."


In [9]:
df['text'][0]

"Bright and joyful is the morn  \r\nFor us to a Child is born  \r\nFrom the highest realms of Heaven  \r\nUnto us a Son is given.  \r\n  \r\nOn His shoulders He shall bear  \r\nPower and majesty, and wear  \r\nOn His vesture and His thigh  \r\nNames most awful, names most high.  \r\n  \r\nWonderful in counsel be  \r\nChrist, the incarnate Deity  \r\nSire of ages, ne'er to cease  \r\nKing of kings, and Prince of peace.  \r\n  \r\nCome and worship at His feet  \r\nYield to Him the homage meet  \r\nFrom the manger to the throne  \r\nHomage due to God alone...\r\n\r\n"

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

: 

In [ ]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

: 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

: 

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.01641853, 0.07843272, ..., 0.        , 0.02680253,
       0.        ])

In [21]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [22]:
recommendation("Waiting For The Man")

["Billy's Bones",
 'Mary Mary',
 'Street Of Dreams',
 'My Little Corner Of The World',
 'Train',
 'From A Whisper To A Scream',
 'Mister Man',
 'Freak Me Out',
 'All We Got Left Is The Beat',
 'Waiting For Your Love',
 'Man Of Our Times',
 'Heavy Metal Lover',
 "It's Not True",
 'Back On The Corner',
 'Desperados Waiting For A Train',
 'Acid Bubble',
 'A Man Is In Love',
 'Better Man',
 'It Gets Me Where I Want To Go',
 'So This Is Love?']

In [ ]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))

: 